In [0]:
import pandas as pd
import spacy

In [0]:
dados_treino = pd.read_csv("/content/drive/My Drive/curso_word2vec/treino.csv")
dados_treino.sample(5)

,title,text,date,category,subcategory,link
28222,Buscar salvadores que nos defendam com muros é...,"Aludindo ao novo presidente dos EUA, Donald Tr...",2017-01-21,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/01/185...
59603,Edu Dracena entra em acordo com a diretoria e ...,"O zagueiro Edu Dracena, 33, não é mais jogador...",2015-01-15,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/01/1...
43458,Seleção não lamenta adiamento de clássico cont...,"O preparador físico da seleção brasileira, Fáb...",2015-12-11,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/11/1...
76295,Bomba da 2ª Guerra Mundial provoca retirada de...,Cerca de 54 mil pessoas serão retiradas de sua...,2016-12-25,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/12/184...
18099,"Se os anos 70 foram de David Bowie, os 80 fora...","Com as mortes de David Bowie e Prince, a músic...",2016-04-22,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/04...


In [0]:
#Rodas apenas uma vez
!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 89.9MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=e6686ba775d01478f4e0e215883920b63ad81fd819ef08daf6ceeaba8f527e86
  Stored in directory: /tmp/pip-ephem-wheel-cache-tsi63j7g/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [0]:
nlp = spacy.load("pt_core_news_sm")

In [0]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)

In [0]:
doc

Rio de Janeiro é uma cidade maravilhosa

In [0]:
type(doc[2])

spacy.tokens.token.Token

In [0]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [0]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

In [0]:
texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [0]:
from time import time

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        n_process = -1)]

tf = time() - t0

print(tf/60)

3.3800423622131346


In [0]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron e le pen a o turno frança revés siglas ...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço e alckmin anuncia que...
4,queda maio a atividade econômica sobe junho bc


In [0]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [0]:
w2v_modelo

In [0]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
86113


In [0]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [0]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2020-05-28 13:18:49,960 : - collecting all words and their counts
2020-05-28 13:18:49,962 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-28 13:18:49,978 : - PROGRESS: at sentence #5000, processed 34716 words, keeping 10129 word types
2020-05-28 13:18:49,995 : - PROGRESS: at sentence #10000, processed 69298 words, keeping 14909 word types
2020-05-28 13:18:50,011 : - PROGRESS: at sentence #15000, processed 103841 words, keeping 18223 word types
2020-05-28 13:18:50,029 : - PROGRESS: at sentence #20000, processed 138620 words, keeping 20969 word types
2020-05-28 13:18:50,042 : - PROGRESS: at sentence #25000, processed 173257 words, keeping 23410 word types
2020-05-28 13:18:50,059 : - PROGRESS: at sentence #30000, processed 207976 words, keeping 25453 word types
2020-05-28 13:18:50,074 : - PROGRESS: at sentence #35000, processed 242567 words, keeping 27263 word types
2020-05-28 13:18:50,087 : - PROGRESS: at sentence #40000, processed 277254 words, keeping 2899

In [0]:
dir(w2v_modelo)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [0]:
w2v_modelo.corpus_count

86113

In [0]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs = 30)

2020-05-28 14:00:39,529 : - training model with 3 workers on 13006 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2
2020-05-28 14:00:40,577 : - EPOCH 1 - PROGRESS: at 65.27% examples, 316221 words/s, in_qsize 5, out_qsize 0
2020-05-28 14:00:41,038 : - worker thread finished; awaiting finish of 2 more threads
2020-05-28 14:00:41,058 : - worker thread finished; awaiting finish of 1 more threads
2020-05-28 14:00:41,068 : - worker thread finished; awaiting finish of 0 more threads
2020-05-28 14:00:41,069 : - EPOCH - 1 : training on 597929 raw words (503079 effective words) took 1.5s, 328984 effective words/s
2020-05-28 14:00:42,094 : - EPOCH 2 - PROGRESS: at 61.92% examples, 306879 words/s, in_qsize 5, out_qsize 0
2020-05-28 14:00:42,614 : - worker thread finished; awaiting finish of 2 more threads
2020-05-28 14:00:42,628 : - worker thread finished; awaiting finish of 1 more threads
2020-05-28 14:00:42,636 : - worker thread finished; awaiting finish of 0 more t

(15087107, 17937870)

In [0]:
w2v_modelo.wv.most_similar("google")

2020-05-28 14:03:55,170 : - precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('apple', 0.599015474319458),
 ('facebook', 0.5608713626861572),
 ('uber', 0.519279956817627),
 ('software', 0.48991674184799194),
 ('amazon', 0.47898781299591064),
 ('waze', 0.46671685576438904),
 ('pfizer', 0.4656011164188385),
 ('buffett', 0.46015262603759766),
 ('tesla', 0.45086461305618286),
 ('yahoo', 0.44927820563316345)]

In [0]:
w2v_modelo.wv.most_similar("microsoft")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('unilever', 0.5995159149169922),
 ('amazon', 0.5668582320213318),
 ('tesla', 0.5598306059837341),
 ('sky', 0.5552534461021423),
 ('lego', 0.5537877082824707),
 ('walmart', 0.5281527042388916),
 ('buffett', 0.5227382779121399),
 ('braskem', 0.5154078006744385),
 ('spotify', 0.5140732526779175),
 ('moma', 0.5056723952293396)]

In [0]:
w2v_modelo.wv.most_similar("barcelona")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('barça', 0.6157211065292358),
 ('psg', 0.5982396602630615),
 ('bayern', 0.5829380750656128),
 ('chelsea', 0.5676937103271484),
 ('juventus', 0.5573890209197998),
 ('leicester', 0.5558568239212036),
 ('madrid', 0.5536109805107117),
 ('liverpool', 0.5416867733001709),
 ('celta', 0.5397297143936157),
 ('united', 0.5392258167266846)]

In [0]:
w2v_modelo.wv.most_similar("messi")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('suárez', 0.6155792474746704),
 ('cristiano', 0.5407193899154663),
 ('barcelona', 0.5369583368301392),
 ('cavani', 0.5257033109664917),
 ('barça', 0.5230746269226074),
 ('neymar', 0.516278862953186),
 ('enrique', 0.5135352611541748),
 ('tevez', 0.5103322267532349),
 ('psg', 0.5058168172836304),
 ('ronaldo', 0.5011476874351501)]

In [0]:
w2v_modelo.wv.most_similar("gm")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('chrysler', 0.7064881324768066),
 ('volks', 0.687446653842926),
 ('honda', 0.684893012046814),
 ('toyota', 0.6765934824943542),
 ('embraer', 0.6621898412704468),
 ('braskem', 0.6262644529342651),
 ('dow', 0.6173127293586731),
 ('volkswagen', 0.606776237487793),
 ('tesla', 0.5979815721511841),
 ('inbev', 0.5939527153968811)]

In [0]:
#Treinamento do modelo Skip-Gram
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens, 
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30)

2020-05-28 14:52:17,156 : - collecting all words and their counts
2020-05-28 14:52:17,157 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-28 14:52:17,177 : - PROGRESS: at sentence #5000, processed 34716 words, keeping 10129 word types
2020-05-28 14:52:17,192 : - PROGRESS: at sentence #10000, processed 69298 words, keeping 14909 word types
2020-05-28 14:52:17,206 : - PROGRESS: at sentence #15000, processed 103841 words, keeping 18223 word types
2020-05-28 14:52:17,221 : - PROGRESS: at sentence #20000, processed 138620 words, keeping 20969 word types
2020-05-28 14:52:17,242 : - PROGRESS: at sentence #25000, processed 173257 words, keeping 23410 word types
2020-05-28 14:52:17,256 : - PROGRESS: at sentence #30000, processed 207976 words, keeping 25453 word types
2020-05-28 14:52:17,271 : - PROGRESS: at sentence #35000, processed 242567 words, keeping 27263 word types
2020-05-28 14:52:17,288 : - PROGRESS: at sentence #40000, processed 277254 words, keeping 2899

(15087384, 17937870)

In [0]:
w2v_modelo_sg.wv.most_similar("google")

2020-05-28 14:57:16,266 : - precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('apple', 0.469560444355011),
 ('reguladores', 0.41687679290771484),
 ('android', 0.3923983573913574),
 ('buffett', 0.37206804752349854),
 ('patentes', 0.3687942624092102),
 ('yahoo', 0.3645310401916504),
 ('verizon', 0.35935860872268677),
 ('waze', 0.353318452835083),
 ('concorda', 0.3500727415084839),
 ('anunciantes', 0.3462163209915161)]

In [0]:
w2v_modelo.wv.most_similar("google")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('apple', 0.599015474319458),
 ('facebook', 0.5608713626861572),
 ('uber', 0.519279956817627),
 ('software', 0.48991674184799194),
 ('amazon', 0.47898781299591064),
 ('waze', 0.46671685576438904),
 ('pfizer', 0.4656011164188385),
 ('buffett', 0.46015262603759766),
 ('tesla', 0.45086461305618286),
 ('yahoo', 0.44927820563316345)]

In [0]:
w2v_modelo_sg.wv.most_similar("gm")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('metalúrgicos', 0.56312096118927),
 ('motors', 0.5179386734962463),
 ('honda', 0.4936729967594147),
 ('audi', 0.48176562786102295),
 ('airbags', 0.48033347725868225),
 ('airbag', 0.47230666875839233),
 ('compartilhamento', 0.4702358543872833),
 ('fiat', 0.4654811918735504),
 ('bmw', 0.4644731879234314),
 ('mitsubishi', 0.4637683629989624)]

In [0]:
w2v_modelo.wv.most_similar("gm")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('chrysler', 0.7064881324768066),
 ('volks', 0.687446653842926),
 ('honda', 0.684893012046814),
 ('toyota', 0.6765934824943542),
 ('embraer', 0.6621898412704468),
 ('braskem', 0.6262644529342651),
 ('dow', 0.6173127293586731),
 ('volkswagen', 0.606776237487793),
 ('tesla', 0.5979815721511841),
 ('inbev', 0.5939527153968811)]

In [0]:
w2v_modelo.wv.save_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_skipgram.txt", binary=False)

2020-05-28 15:06:32,585 : - storing 13006x300 projection weights into /content/drive/My Drive/curso_word2vec/modelo_cbow.txt
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-05-28 15:06:35,478 : - storing 13006x300 projection weights into /content/drive/My Drive/curso_word2vec/modelo_skipgram.txt
